<a href="https://colab.research.google.com/github/anjaspangestu/CCL/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import cv2
import numpy as np
import glob
import os
import shutil


def connected_component_label(filepath):
    tumor = False
    kernel1 = np.ones((3, 3), np.uint8)
    kernel2 = np.ones((5, 5), np.uint8)
    kernel3 = np.ones((11, 11), np.uint8)

    n = cv2.imread(filepath, 0)
    # Thresholding (t == 98)
    n = cv2.threshold(n, 96, 255, cv2.THRESH_BINARY)[1]
    mask = cv2.morphologyEx(n, cv2.MORPH_OPEN, kernel1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel3)
    connectivity = 4
    # Start CCL
    num_labels, labels = cv2.connectedComponents(mask)
    result = cv2.connectedComponentsWithStats(mask, connectivity, cv2.CV_32S)
    num = result[0]
    stats = result[2]

    label_hue = np.uint8(179 * labels / np.max(labels))
    blank_ch = 255 * np.ones_like(label_hue)
    ccl_img = cv2.merge([label_hue, blank_ch, blank_ch])
    ccl_img = cv2.cvtColor(ccl_img, cv2.COLOR_HSV2BGR)
    # background black
    ccl_img[label_hue == 0] = 0

    # print("Jumlah label: ", num_labels)
    for label in range(1, num):
        x = stats[label, cv2.CC_STAT_LEFT]
        y = stats[label, cv2.CC_STAT_TOP]
        w = stats[label, cv2.CC_STAT_WIDTH]
        h = stats[label, cv2.CC_STAT_HEIGHT]
        area = stats[label, cv2.CC_STAT_AREA]
        if w > 30 and h > 30:
            if w < 120 and h < 120 and area > 400:
                cv2.rectangle(ccl_img, (x, y), (x + w, y + h), (0, 255, 0), 1)
                tumor = True
    return ccl_img, tumor


def deleteDir(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete file on %s. Info: %s' % (file_path, e))


#/content/drive/MyDrive/Dataset/Glioma merupakan lokasi dari penyimpanan dataset pada gDrive, bisa diubah sesuai data yg tersimpan pada folder tertentu
sourcePath = glob.glob('/content/drive/MyDrive/Dataset/Glioma' + '/*.png')
targetedPath = '/content/drive/MyDrive/Dataset/Output'
cv_img = []
detected = 0
notDetected = 0
if os.path.exists(targetedPath):
    print("folder found.")
    deleteDir(targetedPath)
    i = 0
    for img in sourcePath:
        #ubah 20 untuk memproses n total dari dataset. Contoh n = 100, maka akan memproses 100 dataset
        if i <= len(sourcePath):
          i += 1
          print("processing data - ", i, 'filepath: ', img)
          output, tumor = connected_component_label(img)
          if tumor:
            print("Tumor detected.");
            detected += 1
          else:
            print("Tumor not detected.");
            notDetected += 1
          cv2.imwrite(f'' + targetedPath + '/output_' + str(i) + '.png', output)
        else:
            break
else:
    print("folder not found.")

print("Total Detected: ", detected, 'from', len(sourcePath))
print("Total Not Detected: ", notDetected, 'from', len(sourcePath))
    


folder found.
processing data -  1 filepath:  /content/drive/MyDrive/Dataset/Glioma/Data_127.png
Tumor detected.
processing data -  2 filepath:  /content/drive/MyDrive/Dataset/Glioma/Data_126.png
Tumor detected.
processing data -  3 filepath:  /content/drive/MyDrive/Dataset/Glioma/Data_125.png
Tumor detected.
processing data -  4 filepath:  /content/drive/MyDrive/Dataset/Glioma/Data_113.png
Tumor detected.
processing data -  5 filepath:  /content/drive/MyDrive/Dataset/Glioma/Data_123.png
Tumor detected.
processing data -  6 filepath:  /content/drive/MyDrive/Dataset/Glioma/Data_112.png
Tumor detected.
processing data -  7 filepath:  /content/drive/MyDrive/Dataset/Glioma/Data_56.png
Tumor detected.
processing data -  8 filepath:  /content/drive/MyDrive/Dataset/Glioma/Data_55.png
Tumor detected.
processing data -  9 filepath:  /content/drive/MyDrive/Dataset/Glioma/Data_25.png
Tumor detected.
processing data -  10 filepath:  /content/drive/MyDrive/Dataset/Glioma/Data_24.png
Tumor detected.